The notebook will loaded the pre-trained model and make the prediction

In [0]:
#Input the model file names here
filename_model = "model_toxic_comment1.json"
filename_weight = "model_toxic_com.h5"


In [2]:
from google.colab import files

uploaded_json = files.upload()


Saving model_toxic_comment1.json to model_toxic_comment1 (1).json


In [3]:
uploaded_h5 = files.upload()

Saving model_toxic_com.h5 to model_toxic_com (1).h5


In [4]:
!git clone https://github.com/tonyyeung99/toxic_comments_data.git

fatal: destination path 'toxic_comments_data' already exists and is not an empty directory.


In [5]:
!ls

datalab			model_toxic_comment1 (1).json  toxic_comments_data
model_toxic_com (1).h5	model_toxic_comment1.json      toxic_comment_submit.csv
model_toxic_com.h5	model_toxic.h5


In [6]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [0]:
test = pd.read_csv('./toxic_comments_data/test.csv')
train = pd.read_csv('./toxic_comments_data/train.csv')

In [8]:
test.isnull().any()

id              False
comment_text    False
dtype: bool

In [9]:
print(len(test))

153164


In [0]:

list_sentences_test = test["comment_text"]
list_sentences_train = train["comment_text"]


In [0]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)


In [0]:
maxlen = 200
X_t = pad_sequences(list_tokenized_test, maxlen=maxlen)


In [0]:
from keras.models import model_from_json
import io
# loaded_model_json = json_file.read()
# json_file.close()
loaded_model = model_from_json(uploaded_json[filename_model])


In [14]:
f = open("model_toxic.h5","wb+")
f.write(uploaded_h5[filename_weight])
f.close

<function BufferedRandom.close>

In [0]:
loaded_model.load_weights('model_toxic.h5')


In [0]:
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [17]:
result = loaded_model.predict(X_t, verbose=1)


print(result)


153164/153164 [==============================] - 273s 2ms/step


[[9.97153997e-01 3.54507416e-01 9.63909328e-01 7.05523267e-02
  8.99317026e-01 2.77550071e-01]
 [7.89372774e-04 2.48586275e-07 7.86491801e-05 2.94651863e-06
  1.73644021e-05 1.10216515e-05]
 [1.78701442e-03 5.62350624e-06 3.14650883e-04 2.43296199e-05
  9.64407154e-05 5.36826556e-05]
 ...
 [3.90446425e-04 6.34174739e-08 5.00906754e-05 6.52309325e-07
  8.28824159e-06 4.76136029e-06]
 [9.69497580e-03 3.10708992e-06 8.84907262e-04 3.09942261e-05
  3.13416211e-04 2.11338775e-04]
 [9.62549448e-01 9.11382679e-03 7.51952112e-01 2.24721944e-03
  3.62438977e-01 2.39719395e-02]]


In [0]:
class_col = pd.DataFrame({'toxic':result[:,0].tolist(), 'severe_toxic':result[:,1].tolist(), 'obscene':result[:,2].tolist(), 'threat':result[:,3].tolist(), 'insult':result[:,4].tolist(), 'identity_hate':result[:,5].tolist()})

In [0]:
id_col = test["id"]


In [0]:
#This line 
frames = [id_col, class_col]

#frames = [test, class_col]

merged = pd.concat(frames, axis=1)

In [21]:
pd.set_option('max_colwidth',402)
batch_line = 18
merged[(batch_line-1) * 10: 10*batch_line]

,id,identity_hate,insult,obscene,severe_toxic,threat,toxic
170,00441d96a027ea2d,0.150128,0.367722,0.213246,4.487315e-02,0.082848,0.785949
171,004515df90b06ef9,0.000159,0.000268,0.000819,1.570573e-05,0.000067,0.004698
172,004540f1a3bba137,0.000036,0.000063,0.000233,1.546398e-06,0.000011,0.001594
173,00463993d4325375,0.000029,0.000051,0.000198,1.503072e-06,0.000010,0.001312
174,004651d6884295bf,0.055014,0.630271,0.529611,2.169803e-02,0.010043,0.967748
175,0046db897eaa12e7,0.139210,0.830962,0.910333,1.926203e-01,0.023680,0.995804
176,0046f41333aebfe7,0.000038,0.000060,0.000193,7.979321e-07,0.000010,0.002174
177,0047582c40509d2d,0.000085,0.000247,0.000468,8.984953e-07,0.000015,0.010374
178,004793abaf67dd8e,0.069577,0.372942,0.260844,2.106522e-02,0.019515,0.859263
179,004902b29a55acae,0.011597,0.054395,0.170699,9.227141e-04,0.001464,0.614936


In [0]:
#pd.set_option('max_colwidth',200)
#merged.loc[merged["obscene"] > 0.5]
  
  


In [22]:
print(loaded_model.metrics_names)

['loss', 'acc']


In [0]:
merged.to_csv('toxic_comment_submit.csv', sep=',', index = False, float_format='%.1f', columns=["id","toxic","severe_toxic","obscene","threat","insult", "identity_hate" ])

In [24]:
!ls

datalab			model_toxic_comment1 (1).json  toxic_comments_data
model_toxic_com (1).h5	model_toxic_comment1.json      toxic_comment_submit.csv
model_toxic_com.h5	model_toxic.h5


In [0]:
files.download('toxic_comment_submit.csv') 